### Prepare Test Data

In [2]:
import math
import pandas as pd
import numpy as np
from datetime import date

In [2]:
df = pd.read_csv('test.csv')
df.head()

,id,artists,name,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,release_date,speechiness,tempo,valence
0,6IfByzdeGt3iIsakTp16vz,['Amanda Miguel'],Ámame una Vez Más,1996,0.5210,0.668,266173,0.452,0,0.000009,9,0.0652,-9.318,1,1996-01-01,0.0314,87.967,0.434
1,17TO3XH1slhDyqnwypne06,['Joe Cocker'],Let It Be,1969,0.2300,0.468,300560,0.280,0,0.005360,0,0.1200,-11.602,1,1969-11-01,0.0291,108.348,0.346
2,5chxjST0MfWZPsaHrWgiqe,['Los Bukis'],Sabes,1984,0.0425,0.315,169827,0.543,0,0.000019,1,0.1420,-11.942,1,1984-01-01,0.0352,158.734,0.423
3,4Db6PxFIiK1a6nCEJABTHk,['Depeche Mode'],Master and Servant,1985,0.1050,0.584,229360,0.866,0,0.001810,10,0.0383,-17.319,0,1985-11-11,0.0401,127.783,0.695
4,3MPKTMJIxlORqcxw542HHK,"['Naughty By Nature', 'Zhané']",Jamboree (feat. Zhané),1999,0.0344,0.849,213907,0.635,1,0.000000,0,0.0820,-9.425,1,1999-04-27,0.2660,92.003,0.856


In [3]:
df_ap = pd.read_csv('data/art_pop.csv')
df_ap.head()

,artist,popularity
0,Erroll Garner,11.144737
1,Maison & Dragen,0.000000
2,Yoel Lerner,36.500000
3,Mazi Cohen,36.500000
4,Barbra Streisand,33.766234


In [9]:
def getArtistPop(artists):
    sum = 0
    qty = 0
    txts = eval(artists)
    for txt in txts:
        if df_ap[df_ap.artist == txt]['popularity'].count() > 0:
            sum += df_ap[df_ap.artist == txt]['popularity'].mean()
        else:
            sum += df_ap[df_ap.artist == 'IMPUTE_VALUE']['popularity'].mean()
        qty += 1
    if qty == 0:
        qty = 1
    return (sum / qty)

def getArtistQty(artists):
    return len(eval(artists))

def getNameLen(name):
    return len(name)

In [5]:
df['artists_popularity'] = df.apply(lambda row : getArtistPop(row['artists']), axis = 1)
df['artists_n']          = df.apply(lambda row : getArtistQty(row['artists']), axis = 1)
df['name_len']           = df.apply(lambda row : getNameLen(row['name']), axis = 1)

In [6]:
df_lan = pd.read_csv('data/test_name_lan.csv')
df['name_lan'] = df_lan['name_lan']

In [7]:
df = df[['year', 'acousticness', 'duration_ms', 'explicit', 'instrumentalness', 
         'key', 'loudness', 'mode', 'tempo', 'valence', 'artists_popularity',
         'artists_n', 'name_len', 'name_lan', 'liveness']]
df.head()

,year,acousticness,duration_ms,explicit,instrumentalness,key,loudness,mode,tempo,valence,artists_popularity,artists_n,name_len,name_lan,liveness
0,1996,0.5210,266173,0,0.000009,9,-9.318,1,87.967,0.434,44.538462,1,17,3,0.0652
1,1969,0.2300,300560,0,0.005360,0,-11.602,1,108.348,0.346,38.264706,1,9,1,0.1200
2,1984,0.0425,169827,0,0.000019,1,-11.942,1,158.734,0.423,42.500000,1,5,2,0.1420
3,1985,0.1050,229360,0,0.001810,10,-17.319,0,127.783,0.695,34.288462,1,18,9,0.0383
4,1999,0.0344,213907,1,0.000000,0,-9.425,1,92.003,0.856,45.650000,2,22,1,0.0820


### Create the final model

In [8]:
final = pd.read_csv('data/train_features.csv')
y = final.pop('popularity').values
X = final.values

In [10]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=15, criterion='mse', random_state=None)
model = model.fit(X, y)

In [11]:
X_test = df.values
df['popularity'] = model.predict(X_test)
df.head()

,year,acousticness,duration_ms,explicit,instrumentalness,key,loudness,mode,tempo,valence,artists_popularity,artists_n,name_len,name_lan,liveness,popularity
0,1996,0.5210,266173,0,0.000009,9,-9.318,1,87.967,0.434,44.538462,1,17,3,0.0652,44.852046
1,1969,0.2300,300560,0,0.005360,0,-11.602,1,108.348,0.346,38.264706,1,9,1,0.1200,39.005634
2,1984,0.0425,169827,0,0.000019,1,-11.942,1,158.734,0.423,42.500000,1,5,2,0.1420,40.701383
3,1985,0.1050,229360,0,0.001810,10,-17.319,0,127.783,0.695,34.288462,1,18,9,0.0383,34.295578
4,1999,0.0344,213907,1,0.000000,0,-9.425,1,92.003,0.856,45.650000,2,22,1,0.0820,46.131429


In [12]:
sub = pd.read_csv('test.csv')
sub = sub[['id']]
sub['popularity'] = df['popularity']
sub.head()

,id,popularity
0,6IfByzdeGt3iIsakTp16vz,44.852046
1,17TO3XH1slhDyqnwypne06,39.005634
2,5chxjST0MfWZPsaHrWgiqe,40.701383
3,4Db6PxFIiK1a6nCEJABTHk,34.295578
4,3MPKTMJIxlORqcxw542HHK,46.131429


In [13]:
sub.to_csv('data/submision.csv', index = False)

### Real Word Validation

In [3]:
final = pd.read_csv('data/train_features.csv')
y = final.pop('popularity').values
X = final.values

In [4]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=15, criterion='mse', random_state=None)
model = model.fit(X, y)

In [21]:
test = pd.DataFrame({
    'id'               : ['00001',             '00002'],
    'artists'          : ["['Justin Bieber']", "['Imagine Dragons']"],
    'name'             : ['Hold On',           'Follow You'],
    'year'             : [2021,                2021],
    'acousticness'     : [0.1,                 0.1],
    'danceability'     : [0.7,                 0.4],
    'duration_ms'      : [304800,              240000],
    'energy'           : [0.5,                 0.7],
    'explicit'         : [0,                   0],
    'instrumentalness' : [0.0,                 0.1],
    'key'              : [5,                   0],
    'liveness'         : [0.2,                 0.001],
    'loudness'         : [-0.3,                -4.0],
    'mode'             : [0,                   1],
    'release_date'     : ["2021",              "2021"],
    'speechiness'      : [0.2,                 0.09],
    'tempo'            : [50.0,                130.0],
    'valence'          : [0.5,                 0.4]
    })

In [22]:
df_ap = pd.read_csv('data/art_pop.csv')

In [23]:
test['artists_popularity'] = test.apply(lambda row : getArtistPop(row['artists']), axis = 1)
test['artists_n']          = test.apply(lambda row : getArtistQty(row['artists']), axis = 1)
test['name_len']           = test.apply(lambda row : getNameLen(row['name']), axis = 1)

In [29]:
def getLanguage(name):
    lans = {'en':1, 'de':2, 'es':3, 'it':4, 'fr':5, 'pt':6, 'nl':7, 'tl':8, 'af':9,
            'id':10, 'no':11, 'ca':12, 'so':13, 'da':14, 'cy':15, 'ro':16, 'sw':17,
            'fi':18, 'et':19, 'sv':20, 'tr':21, 'pl':22, 'sl':23, 'lt':24, 'hu':25,
            'hr':26, 'sk':27, 'vi':28, 'ko':29, 'cs':30, 'sq':31, 'ja':32, 'number':33,
            'lv':34, 'other':35, 'zh-cn':36, 'ru':37, 'th':38, 'zh-tw':39, 'he':40, 'mk':41,
            'bg':42, 'ar':43, 'uk':44, 'fa':45, 'el':46, 'bn':47, 'gu':48, 'hi':49, 'kn':50,
            'ml':51, 'mr':52, 'ne':53, 'pa':54, 'ta':55, 'te':56, 'ur':57}
    
    if (name.isnumeric()):
        res = "number"
    else:
        try:
            res = detect(name)
        except:
            res = "other"
    return lans[res]

In [30]:
test['name_lan'] = test.apply(lambda row : getLanguage(row['name']), axis = 1)

In [31]:
test2 = test[['year', 'acousticness', 'duration_ms', 'explicit', 'instrumentalness', 
             'key', 'loudness', 'mode', 'tempo', 'valence', 'artists_popularity',
             'artists_n', 'name_len', 'name_lan', 'liveness']]
test2.head()

,year,acousticness,duration_ms,explicit,instrumentalness,key,loudness,mode,tempo,valence,artists_popularity,artists_n,name_len,name_lan,liveness
0,2021,0.1,304800,0,0.0,5,-0.3,0,50.0,0.5,61.866667,1,7,35,0.200
1,2021,0.1,240000,0,0.1,0,-4.0,1,130.0,0.4,62.838710,1,10,35,0.001


In [28]:
X_test = test2.values
y = model.predict(X_test)
test['popularity'] = y
test[['artists', 'name', 'popularity']].head()

,artists,name,popularity
0,['Justin Bieber'],Hold On,37.475333
1,['Imagine Dragons'],Follow You,50.637072
